In [1]:
%load_ext autoreload
%autoreload 2
import sys
import gc

from multiprocessing import cpu_count, Pool
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.utils.data as data_utils

from tqdm import tqdm

sys.path.append('../')
from wiki.utils import clock
from wiki import rnn, rnn_predict, newphet, val
%matplotlib inline

In [2]:
torch.manual_seed(1)
torch.cuda.set_device(0)

In [3]:
batch_size = 1024
pred_len = 62

In [4]:
base_dir = '../data/'
train_df = pd.read_csv(base_dir+'train_2.csv').fillna(0)

In [5]:
X = train_df.drop('Page', axis=1).values
X, scaler = rnn.scale_values(X)

In [6]:
X[:,-2*pred_len-430:-2*pred_len,:].shape

(145063, 430, 1)

In [7]:
trainloader = data_utils.DataLoader(
    data_utils.TensorDataset(
        torch.from_numpy(X[:,-2*pred_len-430:-2*pred_len,:]).float(),
        torch.from_numpy(X[:,-2*pred_len:-pred_len,:]).float()
    ),
    batch_size=batch_size, shuffle=False
)

In [8]:
valloader = data_utils.DataLoader(
    data_utils.TensorDataset(
        torch.from_numpy(X[:,:-pred_len,:]).float(),
        torch.from_numpy(X[:,-pred_len:,:]).float()
    ),
    batch_size=batch_size, shuffle=False
)

In [9]:
loss_func = nn.L1Loss()
model = rnn.RNN(loss_func=loss_func).cuda()

In [60]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
save_best_path = base_dir+'rnn_stage2_v2_lr1.mdl'
with clock():
    model.fit(trainloader, valloader, optimizer=optimizer, num_epochs=20, save_best_path=save_best_path)


EPOCH 1
Elapsed time 3.493138074874878 seconds
Elapsed time 3.4933879375457764 seconds


RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1503970438496/work/torch/lib/THC/generic/THCStorage.cu:66

In [ ]:
save_best_path = base_dir+'rnn_stage2_v2_lr1.mdl'
model.load_state_dict(torch.load(save_best_path))

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
save_best_path = base_dir+'rnn_stage2_v2_lr2.mdl'
with clock():
    model.fit(trainloader, valloader, optimizer=optimizer, num_epochs=10, save_best_path=save_best_path)

In [10]:
model = rnn.RNN().cuda()
save_best_path = base_dir+'rnn_stage2_v2_lr2.mdl'
model.load_state_dict(torch.load(save_best_path))

In [11]:
outputs, targets, sequences = model.predict(valloader)

In [12]:
_, predictions = rnn_predict.combine_prediction_data(outputs, targets, sequences)

In [13]:
base_dir = '../data/'
train_df = pd.read_csv(base_dir+'train_2.csv')
X = train_df.drop('Page', axis=1).values

In [14]:
predictions = scaler.inverse_transform(predictions.T).T
true = X

In [15]:
smapes = val.smape(true[:,-60:], predictions[:,-60:], axis=1)
smapes_clipped = val.smape(true[:,-60:], predictions[:,-60:].round().clip(0), axis=1)

../wiki/val.py:8: RuntimeWarning: Mean of empty slice
  return np.nanmean(diff) if axis is None else np.nanmean(diff, axis=axis)
../wiki/val.py:6: RuntimeWarning: invalid value encountered in true_divide
  diff = np.abs(y_true - y_pred) / denominator


In [16]:
np.nanmean(smapes), np.nanmean(smapes_clipped)

(45.292513613168069, 43.341507158056743)

In [17]:
smapes_sequences = val.smape(true[:,-60:], predictions[:,-60:].round().clip(0), axis=1)

../wiki/val.py:6: RuntimeWarning: invalid value encountered in true_divide
  diff = np.abs(y_true - y_pred) / denominator
../wiki/val.py:8: RuntimeWarning: Mean of empty slice
  return np.nanmean(diff) if axis is None else np.nanmean(diff, axis=axis)


In [ ]:
smapes_clipped.shape

In [ ]:
moving = []
for i in range(smapes_clipped.shape[0]):
    moving.append(np.nanmean(smapes_clipped[:i+1]))
moving = np.array(moving)

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
plt.figure(figsize=(10,10))
plt.plot(moving)

In [18]:
median = np.nanmedian(true[:,-49-60:-60], axis=1).reshape(-1,1)

/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/numpy/lib/function_base.py:3858: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)


In [19]:
median.shape

(145063, 1)

In [20]:
smapes_median = val.smape(true[:,-60:], median, axis=0)

../wiki/val.py:6: RuntimeWarning: invalid value encountered in true_divide
  diff = np.abs(y_true - y_pred) / denominator


In [21]:
smapes_median_sequences = val.smape(true[:,-60:], median, axis=1)

../wiki/val.py:6: RuntimeWarning: invalid value encountered in true_divide
  diff = np.abs(y_true - y_pred) / denominator
../wiki/val.py:8: RuntimeWarning: Mean of empty slice
  return np.nanmean(diff) if axis is None else np.nanmean(diff, axis=axis)


In [22]:
np.nanmean(smapes_median_sequences), np.nanmean(smapes_median)

(40.914012766318805, 40.602437280903153)

In [ ]:
m_moving = []
for i in range(smapes_median.shape[0]):
    m_moving.append(np.nanmean(smapes_median[:i+1]))
m_moving = np.array(m_moving)

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(m_moving)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax.grid(True)
ax.plot(moving-m_moving)
plt.show()

In [23]:
joined = np.amin(np.stack([smapes_median_sequences, smapes_sequences]), axis=0) ; joined.shape

(145063,)

In [24]:
np.nanmean(joined)

36.691495991786717

In [25]:
np.nanmean(smapes_median_sequences)

40.914012766318805

In [ ]:
predictions.shape

In [61]:
median.shape

(145063, 1)

In [26]:
median_broadcast = np.broadcast_to(median, (median.shape[0],20))

In [27]:
median_broadcast.shape

(145063, 20)

In [28]:
altered = np.concatenate([predictions[:, -60:-20].round().clip(0), median_broadcast], axis=1)

In [29]:
altered.shape

(145063, 60)

In [39]:
def smape(y_true, y_pred, axis=None):
    # NOTE: should check and make sure that NaNs aren't included
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.nanmean(diff) if axis is None else np.nanmean(diff, axis=axis)

In [48]:
smapes_mixture = val.smape(true[:,-60:], altered.clip(0).round(), axis=0)

../wiki/val.py:6: RuntimeWarning: invalid value encountered in true_divide
  diff = np.abs(y_true - y_pred) / denominator


In [49]:
smapes_mixture

array([ 33.75997856,  33.5718463 ,  34.14765252,  34.6716446 ,
        35.26641258,  36.82959283,  37.42351289,  37.48124897,
        37.39910672,  36.73011972,  37.24271052,  38.09589274,
        39.57001348,  42.4660999 ,  39.0900889 ,  38.57240278,
        39.34292811,  39.44770141,  39.53174488,  40.39350974,
        42.03644763,  40.90963832,  41.65159993,  41.41545421,
        40.59399215,  41.13725552,  42.41462607,  43.98385833,
        42.21985028,  42.0853588 ,  41.15100008,  40.27289957,
        41.52639756,  43.00364798,  43.52938365,  41.7944811 ,
        42.43978001,  42.0850265 ,  43.24134373,  43.2592079 ,
        45.01417021,  45.01560437,  41.78960696,  42.65617633,
        41.97121266,  42.51696973,  41.28666745,  46.042614  ,
        45.68030385,  41.97381172,  41.20350479,  44.98076126,
        40.74091158,  41.74467836,  43.54429272,  45.18962698,
        42.00568439,  41.63447463,  41.68535344,  42.23161607])

In [50]:
np.nanmean(smapes_mixture)

40.844891649481468

## 